In [210]:
import implicit
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import csv 
import scipy
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import svds
from implicit.nearest_neighbours import bm25_weight
from implicit import alternating_least_squares

In [48]:
data = []
with open('interactions_30_ch_no_bots') as csvfile:
    datareader = csv.reader(csvfile, delimiter=' ')
    for subreddit, user, comments, _ in datareader:
        data.append([user, subreddit, int(comments)])

In [49]:
data = pd.DataFrame.from_records(data)

In [50]:
data.columns = ['user', 'subreddit', 'comments']

In [51]:
data['user'] = data['user'].astype("category")
data['subreddit'] = data['subreddit'].astype("category")

In [52]:
# create a sparse matrix of all the artist/user/play triples
comments = coo_matrix((data['comments'].astype(float), 
                   (data['subreddit'].cat.codes, 
                    data['user'].cat.codes)))

### Latent Semantic Analysis

In [211]:
subreddit_factors, user_factors = alternating_least_squares(bm25_weight(comments), 20)

This method is deprecated. Please use the AlternatingLeastSquares class instead
100%|██████████| 15.0/15 [00:54<00:00,  3.48s/it]


In [212]:
class TopRelated(object):
    def __init__(self, subreddit_factors):
        norms = np.linalg.norm(subreddit_factors, axis=-1)
        self.factors = subreddit_factors / norms[:, np.newaxis]
        self.subreddits = data['subreddit'].cat.categories.array.to_numpy()

    def get_related(self, subreddit, N=10):
        subredditid = np.where(self.subreddits == subreddit)[0][0]
        scores = self.factors.dot(self.factors[subredditid])
        best = np.argpartition(scores, -N)[-N:]
        best_ = [self.subreddits[i] for i in best]
        return sorted(zip(best_, scores[best]), key=lambda x: -x[1])

In [213]:
top_related = TopRelated(subreddit_factors)

In [224]:
top_related.get_related('deathnote')

[('deathnote', 1.0),
 ('Naruto', 0.9703468),
 ('CodeGeass', 0.9637688),
 ('OnePiece', 0.96334124),
 ('magickaito1412', 0.95928794),
 ('MSRP', 0.95661414),
 ('Jibba', 0.9543929),
 ('chuunibyou', 0.95296353),
 ('ShokugekiNoSoma', 0.9515109),
 ('YoshisIsland', 0.95139694)]